In [1]:
from math import floor 
def append_packets(packet_info, queues, time):
    for i in range(len(packet_info)):
            for sublist in packet_info:
                if sublist[0] == i:
                    whole_packets = floor((time*sublist[1] - sublist[3]))
                    if whole_packets > 0:
                        queues[i].append(0)
                        #print("Appending to queue", i)
                        sublist[3] += 1
                        
def calc_reward(curr_mean_delay_best_effort, current_waiting_time, queues, action):

#   reward for clearing list
    if not any(queues):
        return 25
#   heavily penalize for choosing empty queue
    if (len(queues[action]) == 0):
        return -25
#   penalize for not popping queue 1 when it has a mean delay over 4
    elif current_waiting_time[0] < -4 and action != 0:
        return -10
#   penalize for not popping queue 2 when it has a mean delay over 6
    elif current_waiting_time[1] < -6 and action != 1:
        return -10
    else:
        if curr_mean_delay_best_effort == 0:
            curr_mean_delay_best_effort = current_waiting_time[2]
            return 1
#       reward if model improves mean_delay in best effort
        elif curr_mean_delay_best_effort < current_waiting_time[2]:
            curr_mean_delay_best_effort = current_waiting_time[2]
            return 1
#       penalize slightly if model mean_delay in best effort queue increases
        elif curr_mean_delay_best_effort > current_waiting_time[2]:
            curr_mean_delay_best_effort = current_waiting_time[2]
            reward = -1
        else:
            reward = 1

def add_time(queue):
    for i in range(len(queue)):
        queue[i] -= 1
    
        
def calc_mean_delay(queue):
    # Calculate the length and average waiting time of the current queue
    current_length = len(queue)
    current_waiting_time = np.average(queue) if current_length > 0 else 0.0
    #print(np.average(current_queue), current_length, current_waiting_time )
    current_waiting_time = np.round(current_waiting_time, 1)
    return current_waiting_time

def catch_over_ten_second_mean_delay(waiting_time):
    if waiting_time < -9.9:
        waiting_time = -9.9

In [2]:
import gym
from gym import spaces
import numpy as np
from math import ceil

class TrafficGenerator(gym.Env):
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(3)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        # Queue maximum length set to 100
        self.observation_space = spaces.Discrete(10000000)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.3, 0.4, 0.4]
        self.mean_delay_req = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3, 0] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.curr_mean_delay_best_effort = 0
        self.packet = 1
        self.timeslot = 1
        self.totaltime = self.timeslot
        
        # Initialize the queues
        self.queues = [[], [], []]
      
    
    def step(self, action):  
        self.totaltime += self.timeslot
        #print(self.totaltime)
        #print("~~~~~~~~~~~~~~~")
        #print("Action Chosen: ", action)
       
       # if action < 3 and len(self.queues[action]) > 0:
        #    self.queues[action].pop(0)

        
        #OBSERVING ENVRONEMNT
        observation = []
        
        #append_packets(self.packetInfo, self.queues, self.totaltime)
        
        for i in range(len(self.mean_delay_req)):
            #add_time(self.queues[i])
            # Get the current queue for the packet type
            current_queue = self.queues[i]
            # Calculate the length and average waiting time of the current queue
            current_waiting_time = calc_mean_delay(current_queue)
            observation.append(current_waiting_time)
            
        reward = calc_reward(self.curr_mean_delay_best_effort, observation, self.queues, action)
        
        
        
        
        #UPDATING ENVIRONMENT
        if action < 3 and len(self.queues[action]) > 0:
            self.queues[action].pop(0)
    
        append_packets(self.packetInfo, self.queues, self.totaltime)
        

        observationUpdate = []        
        for i in range(len(self.mean_delay_req)):
            add_time(self.queues[i])
            # Get the current queue for the packet type
            current_queue = self.queues[i]
            # Calculate the length and longest waiting packet of each queue
            current_length = len(current_queue)
            current_max_time = np.min(current_queue) if current_length > 0 else 0.0
            observationUpdate.append([current_length, current_max_time])       
        
        observationUpdate = np.array(observationUpdate)    
        
        info = {"Mean Delay Req 0": self.mean_delay_req[0],
        "Mean Delay Req 1": self.mean_delay_req[1],
        "Mean Delay Req 2": self.mean_delay_req[2],
        "arrival_rate_0": self.arrival_rate[0],
        "arrival_rate_1": self.arrival_rate[1],
        "arrival_rate_2": self.arrival_rate[2]}
        #done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0
        done = (self.totaltime >= 100)
        print(observation)
        for i in range(3):
            catch_over_ten_second_mean_delay(observation[i])
        #print(f"WAITING TIMES {np.array(waiting_times)}")
        #waiting_times = discretize(observationUpdate)
        
        
        
        return observationUpdate, reward, done, info
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        self.packetInfo = [[elem1, elem2, elem3, 0] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.totaltime = 1
        return np.array([[len(self.queues[i]), 0.0] for i in range(len(self.mean_delay_req))])
        
    def render(self):
        pass

In [3]:
#FIFO
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
totalReward = 0
while x < 100:
    #print("~~~~~~~~~")
    #print(type(obs))
    #print(obs)
    #print("Actions to choose form", obs[:3, 1]) 
    action = np.argmin(obs[:3, 1])
    
    if (np.any(obs[:3,0]) and not np.any(obs[:3,1] )):
        action = np.argmax(obs[:3, 0])
    
    #print("chosen", action)
    obs,reward, done, info = env.step(action)
    
    totalReward += reward
    #print("reward", reward)
    
    x = x +1
print(totalReward)

[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, -1.0, -1.0]
[-1.0, 0.0, -2.0]
[-2.0, -1.0, -1.0]
[0.0, -2.0, -2.0]
[-1.0, 0.0, -3.0]
[-2.0, -1.0, -1.0]
[0.0, -2.0, -2.0]
[-1.0, -1.0, -2.0]
[-2.0, -2.0, -2.0]
[0.0, -3.0, -3.0]
[0.0, -1.0, -2.5]
[-1.0, -2.0, -2.0]
[-2.0, -1.0, -2.0]
[-3.0, -2.0, -2.0]
[-1.0, -3.0, -3.0]
[-2.0, -1.0, -2.5]
[-3.0, -2.0, -2.0]
[-1.0, -2.0, -2.0]
[-2.0, -2.0, -3.0]
[-3.0, -3.0, -3.0]
[0.0, -2.5, -2.5]
[-1.0, -2.0, -3.5]
[-2.0, -2.0, -2.0]
[-3.0, -2.0, -3.0]
[-2.5, -3.0, -3.0]
[-2.0, -2.5, -2.5]
[-3.0, -2.0, -3.5]
[-2.5, -2.0, -2.0]
[-2.0, -3.0, -3.0]
[-3.0, -3.0, -4.0]
[-4.0, -2.5, -2.5]
[-1.0, -3.5, -3.5]
[-2.0, -2.0, -3.3]
[-3.0, -3.0, -3.0]
[-2.5, -3.0, -4.0]
[-3.5, -2.5, -2.5]
[-3.0, -3.5, -3.5]
[-2.5, -2.0, -3.3]
[-3.5, -3.0, -3.0]
[-3.0, -4.0, -4.0]
[-4.0, -2.5, -3.7]
[-3.0, -3.5, -3.5]
[-2.0, -3.3, -3.3]
[-3.0, -3.0, -4.3]
[-2.5, -4.0, -4.0]
[-3.5, -2.5, -3.7]
[-4.5, -3.5, -3.5]
[-2.5, -3.3, -3.3]
[-3.5, -3.0, -4.3]
[-4.5, -4.0, -4.0]
[-4.0, -3.7, -3.7]
[-3.0, 